In [76]:
from cnns.utils import clustering_utils as clu
from cnns.utils import cnn_utils as cu
from cnns.core import RankOrderClusterer as roc
from cnns.core import AgglomerativeClusterer as agg
from sklearn.metrics.cluster import homogeneity_completeness_v_measure
from collections import defaultdict
import pandas as pd
import pickle as pkl
import numpy as np

In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
user_id = 1946418
openface_dir = '/Users/babasarala/Desktop/face_clustering/openface_cnn_codes'
cnn_codes_filepath = '%s/%i_cnn_codes.p'%(openface_dir, user_id)
openface_results_dir = '/Users/babasarala/Desktop/OpenFace+Agg+RankOrder_testing'
agg_settings_filepath = '/Users/babasarala/repos/cnns/config/agg_settings_v1.0.ini'
img_urls, X = pkl.load(open(cnn_codes_filepath, 'rb'))

In [79]:
# first-pass - use Agglomerative clustering to find a reasonable, highly homogenous set of clusters

# use agglomerative clustering to start and get an initial set of tight clusters
agg_ = agg.AgglomerativeClusterer(dist_thresh = 0.6, method='average')
agg_.load_data(img_urls, X)
_, cluster_ids = agg_.agg_cluster_features()
cluster_df = agg_.convert_to_dataframe(cluster_ids)
cluster_map, _, _ = agg_.convert_to_maps(cluster_df)
clu.visualize_clusters(cluster_map, '/Users/babasarala/Desktop/%i_clusters.html'%user_id)
C = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    C[cluster_id].append(idx)

In [80]:
roc_ = roc.RankOrderClusterer(t=16,K=9, num_merges=2)
roc_.load_data(img_urls, X)
roc_.load_preexisting_clusters(C) # start this off

In [81]:
cluster_df = roc_.run()
cluster_map, _, _ = roc_.convert_to_maps(cluster_df)
clu.visualize_clusters(cluster_map, '/Users/babasarala/Desktop/%i_hybrid_clusters.html'%user_id)
len(roc_.C)

86